# Variational Method Test

In [1]:
import numpy as np  
import functools
import itertools
import matplotlib.pyplot as plt 
from scipy   import  linalg as LA 
import json

from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import ParameterVector

from qiskit.utils                     import QuantumInstance
from qiskit.opflow                    import Z, I, X, Y
from qiskit.opflow                    import PauliOp, SummedOp

from shor_function  import *
from pVQD           import *
from ansatze        import *
from itertools import permutations

/var/folders/1k/wfwgsdr57xz4qwc3p5r0l92h0000gq/T/ipykernel_82584/1118443837.py:13: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow                    import Z, I, X, Y


In [2]:
potential = lambda x: x**2
potential_neg = lambda x: -x**2
true_sol = lambda x, t: normalize(1/np.sqrt(2)*np.pi**(-1/4)*(2*x**2 - 1)*np.exp(-x**2/2)*np.exp(-1j*5*t))
initial_wave_function = lambda x: true_sol(x, 0)
normalize = lambda x: x / np.linalg.norm(x)
relative_diff = lambda x, y: np.linalg.norm(np.abs(x)**2 - np.abs(y)**2)/np.linalg.norm(np.abs(y)**2)

In [75]:
def para_ansatz(initial_wave_function, x_grid, n, depth, parameters):
    initial_vector = initial_wave_function(x_grid)
    qc = QuantumCircuit(n)
    qc.prepare_state(state=initial_vector)
    
    new_circuit = hweff_ansatz(n,depth,parameters)
    circuit_par = qc & new_circuit
    
#     print(circuit_par)
    
    states = []
    states.append(Statevector.from_instruction(circuit_par))
    
    return states

def algo_ansatz(initial_wave_function, potential, x_grid, n, L, dt, D, order):
    initial_vector = initial_wave_function(x_grid)
    qc = QuantumCircuit(n)
    qc.prepare_state(state=initial_vector)
    
    if order == 1:
        circuit = walsh_quantum_trotter(potential, initial_wave_function, n, L, dt, D, terms_kept=None, verbose=True, gate_count_only=False)
        
    if order == 2:       
        circuit = walsh_quantum_strang(potential, initial_wave_function, n, L, dt, D, terms_kept=None, verbose=True, gate_count_only=False)
        
    if order == 4:    
        circuit = walsh_quantum_4(potential, initial_wave_function, n, L, dt, D, terms_kept=None, verbose=True, gate_count_only=False)
        
    circuit_alg = qc & circuit
    
#     print(circuit_alg)
    
    states = []
    states.append(Statevector.from_instruction(circuit_alg))
    
    return states

In [82]:
# Initialize system parameters for Time-dependent Schrodinger
n       = 3
N       = 2**n
L       = 8
dx      = 2*L/N
x_grid = np.arange(-L, L - dx/2, dx)
# g       = 1.0
dt      = 0.01
n_steps = 1

# Algorithm parameters

ths = 0.999999
depth = 3


### Example circ

ex_params = np.zeros((depth+1)*n +depth*(n-1))
wfn = hweff_ansatz(n,depth,ex_params)


### Shift
shift  = np.array(len(ex_params)*[0.01])

print("Initial shift:",shift)


### Generate the Hamiltonian

## Note: in the hermite example D = 1, i.e., m = 0.5, in the tunneling example D = 1/2, i.e., m = 1
## Note: In the fidelity, we need e^{iHt}, so potential and D are negative


H = walsh_quantum_trotter(potential_neg, initial_wave_function, n, L, dt, D=-1, terms_kept=None, verbose=True, gate_count_only=False)
# H = walsh_quantum_strang(potential_neg, initial_wave_function, n, L, dt, D=-1, terms_kept=None, verbose=True, gate_count_only=False)
# H = walsh_quantum_4(potential_neg, initial_wave_function, n, L, dt, D=-1, terms_kept=None, verbose=True, gate_count_only=False)
# initial_vector = initial_wave_function(x_grid)
# initial_vector = [1,0,0,0]
# all_orderings = list(permutations(initial_vector))
# qc = QuantumCircuit(n)
# qc.initialize(initial_vector)
# H = qc

print(wfn)
# print(H)

### Backend
shots = 8000
backend  = Aer.get_backend('qasm_simulator')
instance = QuantumInstance(backend=backend,shots=shots)

# ### Prepare the observables to measure
obs = {}


Initial shift: [0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01]
     ┌───────┐ ░                  ░ ┌───────┐ ░                  ░ ┌───────┐ ░ »
q_0: ┤ Rx(0) ├─░──■───────────────░─┤ Ry(0) ├─░──■───────────────░─┤ Rx(0) ├─░─»
     ├───────┤ ░  │ZZ(0)          ░ ├───────┤ ░  │ZZ(0)          ░ ├───────┤ ░ »
q_1: ┤ Rx(0) ├─░──■───────■───────░─┤ Ry(0) ├─░──■───────■───────░─┤ Rx(0) ├─░─»
     ├───────┤ ░          │ZZ(0)  ░ ├───────┤ ░          │ZZ(0)  ░ ├───────┤ ░ »
q_2: ┤ Rx(0) ├─░──────────■───────░─┤ Ry(0) ├─░──────────■───────░─┤ Rx(0) ├─░─»
     └───────┘ ░                  ░ └───────┘ ░                  ░ └───────┘ ░ »
«                      ░ ┌───────┐
«q_0: ─■───────────────░─┤ Ry(0) ├
«      │ZZ(0)          ░ ├───────┤
«q_1: ─■───────■───────░─┤ Ry(0) ├
«              │ZZ(0)  ░ ├───────┤
«q_2: ─────────■───────░─┤ Ry(0) ├
«                      ░ └───────┘


/var/folders/1k/wfwgsdr57xz4qwc3p5r0l92h0000gq/T/ipykernel_82584/3519623044.py:51: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  instance = QuantumInstance(backend=backend,shots=shots)


In [83]:
### Initialize the algorithm

# Choose a specific set of parameters
initial_point = None

# Choose the gradient optimizer: 'sgd', 'adam'
opt  = 'sgd'
# Choose how to estimate the gradient on hardware: 'param_shift', 'spsa'
grad = 'param_shift'
# Choose which type of cost function use: 'global', 'local'
cost = 'local'


algo = pVQD(hamiltonian   = H,
			ansatz        = hweff_ansatz,
			ansatz_reps   = depth,
			parameters    = ex_params,
			initial_shift = shift,
			instance      = instance,
			shots         = shots)

para = algo.run(ths,dt,n_steps, 
	     obs_dict      = obs,
	     filename      = 'data/test_results.dat',
	     max_iter      = 50,
	     opt           = opt,
	     cost_fun      = cost,
	     grad          = grad,
	     initial_point = initial_point,
         is_circuit    = True)

ComposedOp([
  OperatorMeasurement(0.16666666666666666 * III
  + 0.16666666666666666 * ZII
  + 0.16666666666666666 * III
  + 0.16666666666666666 * IZI
  + 0.16666666666666666 * III
  + 0.16666666666666666 * IIZ),
  CircuitStateFn(
       ┌──────────┐ ░                        ░ ┌──────────┐ ░            »
  q_0: ┤ Rx(r[0]) ├─░──■─────────────────────░─┤ Ry(r[5]) ├─░──■─────────»
       ├──────────┤ ░  │ZZ(r[3])             ░ ├──────────┤ ░  │ZZ(r[8]) »
  q_1: ┤ Rx(r[1]) ├─░──■──────────■──────────░─┤ Ry(r[6]) ├─░──■─────────»
       ├──────────┤ ░             │ZZ(r[4])  ░ ├──────────┤ ░            »
  q_2: ┤ Rx(r[2]) ├─░─────────────■──────────░─┤ Ry(r[7]) ├─░────────────»
       └──────────┘ ░                        ░ └──────────┘ ░            »
  «                 ░ ┌───────────┐ ░                          ░ ┌───────────┐»
  «q_0: ────────────░─┤ Rx(r[10]) ├─░──■───────────────────────░─┤ Ry(r[15]) ├»
  «                 ░ ├───────────┤ ░  │ZZ(r[13])              ░ ├───────────┤»
  «q

In [84]:
algo_soln = algo_ansatz(initial_wave_function, potential, x_grid, n, L, dt, D = 1, order = 1)
para_soln = para_ansatz(initial_wave_function, x_grid, n, depth, para) 
true_soln = true_sol(x_grid, dt)
diff_algo = relative_diff(algo_soln, true_soln)
diff_para = relative_diff(para_soln, true_soln)

print(algo_soln)
print(para_soln)
print(true_soln)

print(diff_algo)
print(diff_para)

[Statevector([-2.60927661e-04+0.0014423j ,  3.25217094e-04-0.00178354j,
              5.40704163e-03+0.00471209j,  5.58291276e-01+0.09754401j,
             -5.84753863e-01-0.12467298j,  5.58291276e-01+0.09754401j,
              5.40704163e-03+0.00471209j,  3.25217094e-04-0.00178354j],
            dims=(2, 2, 2))]
[Statevector([ 0.00207209+5.96502915e-04j, -0.00239027+1.01557559e-04j,
              0.00516954-4.06931942e-03j,  0.56662982+1.90481324e-05j,
             -0.59918017-4.32596269e-03j,  0.56510814+2.13238017e-02j,
              0.00575798-7.78632300e-04j,  0.00239361+4.30615921e-05j],
            dims=(2, 2, 2))]
[ 9.60803843e-13-4.80802657e-14j  6.45969638e-07-3.23254242e-08j
  6.21241228e-03-3.10879724e-04j  5.65931031e-01-2.83201556e-02j
 -5.97385162e-01+2.98941741e-02j  5.65931031e-01-2.83201556e-02j
  6.21241228e-03-3.10879724e-04j  6.45969638e-07-3.23254242e-08j]
0.0005757731446586792
0.003120428021560391
